In [8]:
import sys
print(sys.executable) 
! /Users/lucasyim/opt/anaconda3/bin/python -m pip install utils
! /Users/lucasyim/opt/anaconda3/bin/python -m pip install pretty_midi


/Users/lucasyim/opt/anaconda3/bin/python
  Using cached pretty_midi-0.2.10.tar.gz (5.6 MB)
  Using cached mido-1.3.0-py3-none-any.whl (50 kB)
  Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Created wheel for pretty-midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592305 sha256=e286edb7d147b47f46ad1914285c194d2fa1b7ce0a1005766b6ca44a80c9805d
  Stored in directory: /Users/lucasyim/Library/Caches/pip/wheels/1f/54/f1/fca57848157fc8f2cbb513575f242d517eb0e44f14e4179a52
Successfully built pretty-midi
  Attempting uninstall: packaging
    Found existing installation: packaging 21.0
    Uninstalling packaging-21.0:
      Successfully uninstalled packaging-21.0


In [9]:
'''
This is the data processing script for POP909:A Pop song Dataset for Music Arrangement Generation
============
It will allow you to quickly process the POP909 Files (Midi) into the Google Magenta's music representation 
    as like [Music Transformer](https://magenta.tensorflow.org/music-transformer) 
            [Performance RNN](https://magenta.tensorflow.org/performance-rnn).

'''



import pickle
import os
import sys
import utils
from processor import MidiEventProcessor
import pretty_midi as pyd
import numpy as np



In [65]:
# Create separate folders for the separated midis

for i in range(1,910):
    num = str(i)
    if i < 10:
        num = "00"+num
    elif i < 100:
        num = "0"+num
    os.mkdir(f"../melody/{num}")
    os.mkdir(f"../chords/{num}")


In [68]:
total = 0
def preprocess_midi(path):
    global total
    data = pyd.PrettyMIDI(path)
    
    main_notes = []
    acc_notes = []
    for ins in data.instruments:
        acc_notes.extend(ins.notes)
    for i in range(len(main_notes)):
        main_notes[i].start = round(main_notes[i].start,2)
        main_notes[i].end = round(main_notes[i].end,2)
    for i in range(len(acc_notes)):
        acc_notes[i].start = round(acc_notes[i].start,2)
        acc_notes[i].end = round(acc_notes[i].end,2)
    main_notes.sort(key = lambda x:x.start)
    acc_notes.sort(key = lambda x:x.start)
    mpr = MidiEventProcessor()
    
    # changed from [main_notes, acc_notes] to main_notes+ acc_notes
    repr_seq = mpr.encode(main_notes + acc_notes)
    total += len(repr_seq)
    return repr_seq


def separate_melody(path):
    global total
    data = pyd.PrettyMIDI(path)
    #print(data.instruments)
    
    melody_midi = pyd.PrettyMIDI()
    chords_midi = pyd.PrettyMIDI()
    
    #piano_program = pyd.instrument_name_to_program('PIANO')
    piano = pyd.Instrument(program=0)
    melody = pyd.Instrument(program=0)
    
    
    # go through each instrument and separate into melody and piano
    for instrument in data.instruments: 
        if instrument.name == "MELODY":
            melody.notes.extend(instrument.notes)
        if instrument.name == "PIANO":
            piano.notes.extend(instrument.notes)

    melody_midi.instruments.append(melody)
    chords_midi.instruments.append(piano)
    #    print(f"instrument: {instrument}: {len(instrument.notes)}")
    #    print("\n\n\n")
    
    print("../melody/"+path[10:-4]+"melody.mid")
    melody_midi.write("../melody/"+path[10:-4]+"melody.mid")
    chords_midi.write("../chords/"+path[10:-4]+"chords.mid")
    
    

def preprocess_pop909_single_midi(midi_root, save_dir):
    save_py = []
    midi_paths = [d for d in os.listdir(midi_root)]
    i = 0
    out_fmt = '{}-{}.data'
    for path in midi_paths:
        #print(' ', end='[{}]'.format(path), flush=True)
        filename = midi_root + path

        if filename[-3:] != "mid":
            continue
        try:
            #data = preprocess_midi(filename)
            print(filename)
            separate_melody(filename)
        except KeyboardInterrupt:
            print(' Abort')
            return
        except EOFError:
            print('EOF Error')
            return
        #save_py.append(data)
    #save_py = np.array(save_py)
    #print(save_py.size)
    #np.save("pop909-event-token.npy", save_py)
            
def preprocess_pop909(midi_root, save_dir):
    save_py = []
    midi_paths = [d for d in os.listdir(midi_root)]
    i = 0
    out_fmt = '{}-{}.data'
    for path in midi_paths:
        # print(' ', end='[{}]'.format(path), flush=True)
        
        try: 
            int(path)
        except:
            continue
        filename = midi_root + path + "/" + path + ".mid"
        # print(filename)
        data = []
        if filename[-3:] != "mid":
            continue
        try:
            # data = preprocess_midi(filename)
            
            separate_melody(filename)
        except KeyboardInterrupt:
            print(' Abort')
            return
        except EOFError:
            print('EOF Error')
            return
        #save_py.append(data)
    #save_py = np.array(save_py, dtype=object)
    #print(save_py.size)
    #np.save("pop909-event-token.npy", save_py)
             
# replace the folder with your POP909 data folder
# not sure if each midi file should be its own npy file or if the entire dataset should be put into one file????????
# Basic structure of how they were originally doing it makes me think one file but not sure as of now
# might make more sense to do 1 at a time since we avoid the problem with ragged arrays

# uncomment based on which approach, single_midi for one file (must come up with new name for each file to not overwrite)
# original for all of them in one file.
#preprocess_pop909_single_midi("../POP909/001/","midi_data/")
preprocess_pop909("../POP909/", "midi_data/")


../melody//797/797melody.mid
../melody//909/909melody.mid
../melody//135/135melody.mid
../melody//307/307melody.mid
../melody//763/763melody.mid
../melody//551/551melody.mid
../melody//338/338melody.mid
../melody//556/556melody.mid
../melody//764/764melody.mid
../melody//300/300melody.mid
../melody//132/132melody.mid
../melody//790/790melody.mid
../melody//569/569melody.mid
../melody//336/336melody.mid
../melody//104/104melody.mid
../melody//560/560melody.mid
../melody//752/752melody.mid
../melody//594/594melody.mid
../melody//907/907melody.mid
../melody//309/309melody.mid
../melody//799/799melody.mid
../melody//593/593melody.mid
../melody//755/755melody.mid
../melody//567/567melody.mid
../melody//103/103melody.mid
../melody//331/331melody.mid
../melody//900/900melody.mid
../melody//558/558melody.mid
../melody//168/168melody.mid
../melody//391/391melody.mid
../melody//533/533melody.mid
../melody//701/701melody.mid
../melody//365/365melody.mid
../melody//157/157melody.mid
../melody//739

../melody//204/204melody.mid
../melody//036/036melody.mid
../melody//658/658melody.mid
../melody//832/832melody.mid
../melody//031/031melody.mid
../melody//203/203melody.mid
../melody//667/667melody.mid
../melody//455/455melody.mid
../melody//693/693melody.mid
../melody//294/294melody.mid
../melody//604/604melody.mid
../melody//436/436melody.mid
../melody//052/052melody.mid
../melody//260/260melody.mid
../melody//099/099melody.mid
../melody//851/851melody.mid
../melody//409/409melody.mid
../melody//267/267melody.mid
../melody//869/869melody.mid
../melody//055/055melody.mid
../melody//431/431melody.mid
../melody//603/603melody.mid
../melody//293/293melody.mid
../melody//856/856melody.mid
../melody//258/258melody.mid
../melody//407/407melody.mid
../melody//635/635melody.mid
../melody//251/251melody.mid
../melody//063/063melody.mid
../melody//097/097melody.mid
../melody//860/860melody.mid
../melody//438/438melody.mid
../melody//894/894melody.mid
../melody//090/090melody.mid
../melody//858

../melody//846/846melody.mid
../melody//283/283melody.mid
../melody//613/613melody.mid
../melody//421/421melody.mid
../melody//879/879melody.mid
../melody//045/045melody.mid
../melody//277/277melody.mid
../melody//884/884melody.mid
../melody//428/428melody.mid
../melody//870/870melody.mid
../melody//087/087melody.mid
../melody//073/073melody.mid
../melody//241/241melody.mid
../melody//625/625melody.mid
../melody//417/417melody.mid
../melody//877/877melody.mid
../melody//279/279melody.mid
../melody//883/883melody.mid
../melody//410/410melody.mid
../melody//622/622melody.mid
../melody//246/246melody.mid
../melody//848/848melody.mid
../melody//074/074melody.mid
../melody//080/080melody.mid
../melody//676/676melody.mid
../melody//444/444melody.mid
../melody//020/020melody.mid
../melody//212/212melody.mid
../melody//682/682melody.mid
../melody//823/823melody.mid
../melody//649/649melody.mid
../melody//685/685melody.mid
../melody//215/215melody.mid
../melody//027/027melody.mid
../melody//443

../melody//728/728melody.mid
../melody//380/380melody.mid
../melody//374/374melody.mid
../melody//146/146melody.mid
../melody//522/522melody.mid
../melody//710/710melody.mid
../melody//179/179melody.mid
../melody//328/328melody.mid
../melody//541/541melody.mid
../melody//773/773melody.mid
../melody//317/317melody.mid
../melody//125/125melody.mid
../melody//787/787melody.mid
../melody//579/579melody.mid
../melody//780/780melody.mid
../melody//122/122melody.mid
../melody//310/310melody.mid
../melody//774/774melody.mid
../melody//546/546melody.mid
../melody//789/789melody.mid
../melody//319/319melody.mid
../melody//584/584melody.mid
../melody//742/742melody.mid
../melody//570/570melody.mid
../melody//114/114melody.mid
../melody//326/326melody.mid
../melody//548/548melody.mid
../melody//321/321melody.mid
../melody//113/113melody.mid
../melody//577/577melody.mid
../melody//745/745melody.mid
../melody//583/583melody.mid
../melody//147/147melody.mid
../melody//375/375melody.mid
../melody//711